# DATA620: Assignment 10 - Document Classification

## Homework Team 3: David Simbandumwe, Eric Lehmphul and Lidiia Tronina

It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.  Here is one example of such data: [UCI Machine Learning Repository: Spambase Data Set](http://archive.ics.uci.edu/ml/datasets/Spambase)
For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).
For more adventurous students, you are welcome to come up a different set of documents that have already been classified , then analyze these documents to predict how new documents should be classified.

### Load Required Packages

In [404]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import random
import nltk
from nltk.corpus import inaugural
from operator import itemgetter
warnings.filterwarnings("ignore")
nltk.download('inagural')
nltk.download('stopwords')

[nltk_data] Error loading inagural: Package 'inagural' not found in
[nltk_data]     index
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lidiiatronina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Choose a corpus of interest for classification.

Presidents’ words matter. For better or worse, presidential rhetoric tells the American people who they are. That's why we decided to look at the inaugural speeches from the freely available library that can be downloaded from the NLTK package. The corpus is a collection of 55 texts, one for each presidential address.
Let's look at available texts in the corpus.

In [405]:
inaugural.fileids()

['1789-Washington.txt',
 '1793-Washington.txt',
 '1797-Adams.txt',
 '1801-Jefferson.txt',
 '1805-Jefferson.txt',
 '1809-Madison.txt',
 '1813-Madison.txt',
 '1817-Monroe.txt',
 '1821-Monroe.txt',
 '1825-Adams.txt',
 '1829-Jackson.txt',
 '1833-Jackson.txt',
 '1837-VanBuren.txt',
 '1841-Harrison.txt',
 '1845-Polk.txt',
 '1849-Taylor.txt',
 '1853-Pierce.txt',
 '1857-Buchanan.txt',
 '1861-Lincoln.txt',
 '1865-Lincoln.txt',
 '1869-Grant.txt',
 '1873-Grant.txt',
 '1877-Hayes.txt',
 '1881-Garfield.txt',
 '1885-Cleveland.txt',
 '1889-Harrison.txt',
 '1893-Cleveland.txt',
 '1897-McKinley.txt',
 '1901-McKinley.txt',
 '1905-Roosevelt.txt',
 '1909-Taft.txt',
 '1913-Wilson.txt',
 '1917-Wilson.txt',
 '1921-Harding.txt',
 '1925-Coolidge.txt',
 '1929-Hoover.txt',
 '1933-Roosevelt.txt',
 '1937-Roosevelt.txt',
 '1941-Roosevelt.txt',
 '1945-Roosevelt.txt',
 '1949-Truman.txt',
 '1953-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1965-Johnson.txt',
 '1969-Nixon.txt',
 '1973-Nixon.txt',
 '1

We have 59 Presidential inaugural speeches. We want to classify presidential speeches as Republican/Democrat. Most presidents have belonged to one of these two parties (except some who were Whigs and National Union), and we can start as early as Andrew Jackson's 1829 speech. It will be interesting to see if the two parties have different word use in their addresses and if we can identify the president's party by word choices. 

In [406]:
party = pd.read_csv('https://raw.githubusercontent.com/dsimband/DATA620_Group3/main/Week10_Assignment/presidents.csv',index_col=0)

party.head(20)

,president,party
year,,
1789,Washington,None
1793,Washington,None
1797,Adams,Federalist
1801,Jefferson,Democrat-Republican
1805,Jefferson,Democrat-Republican
1809,Madison,Democrat-Republican
1813,Madison,Democrat-Republican
1817,Monroe,Democrat-Republican
1821,Monroe,Democrat-Republican


## Data Exploration

There are a total of 152901 words in the corpus.

In [407]:
# Count ALL words
all_words = inaugural.words()
len(all_words)

152901

In [408]:
#Washington's speech
inaugural.words('1789-Washington.txt')

['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', ...]

In [409]:
recent_list = inaugural.fileids()[-10:] 
for text in recent_list:
    word_list = inaugural.words(text)
    word_list = [w.lower() for w in word_list]  # handle the case sensitivity
    unique_words = len(set(word_list))
    print ("For text " + text + ", the number of unique words is", str(unique_words))

For text 1985-Reagan.txt, the number of unique words is 876
For text 1989-Bush.txt, the number of unique words is 754
For text 1993-Clinton.txt, the number of unique words is 604
For text 1997-Clinton.txt, the number of unique words is 727
For text 2001-Bush.txt, the number of unique words is 593
For text 2005-Bush.txt, the number of unique words is 742
For text 2009-Obama.txt, the number of unique words is 900
For text 2013-Obama.txt, the number of unique words is 792
For text 2017-Trump.txt, the number of unique words is 547
For text 2021-Biden.txt, the number of unique words is 783


In [410]:
import pandas as pd
text_data = pd.DataFrame(columns = ['filename','year','length','unique'])
for file in inaugural.fileids():
    word_list = inaugural.words(file)
    word_list = [w.lower() for w in word_list] 
    this_file = pd.DataFrame(data = {"filename":[file], \
                                     "president" : [str(file[5:])], \
                                     "year" : [int(file[:4])], \
                                     "length" : [len(word_list)], \
                                     "unique" : [len(set(word_list))]})
    text_data = text_data.append(this_file, ignore_index=True)


In [411]:
text_data['president'] = text_data['president'].str.replace('.txt','')
president_party = pd.merge(text_data, party, on='president', how='outer')

president_party[-3:]

,filename,length,president,unique,year,party
116,2013-Obama.txt,2369,Obama,792,2013,Democrat
117,2017-Trump.txt,1693,Trump,547,2017,Republican
118,2021-Biden.txt,3104,Biden,783,2021,Democrat


First we need to take all Democrat/Republican speeches and combine them to create one text. We will also remove punctuation and convert everything to lowercase to eliminate duplicate words. Then we can take that and create a list of text segments. Each segment will have a length of 1000 words.

In [412]:
democrat = inaugural.words('1833-Jackson.txt')+inaugural.words('1837-VanBuren.txt')+inaugural.words('1845-Polk.txt')+inaugural.words('1853-Pierce.txt')+inaugural.words('1857-Buchanan.txt')+inaugural.words('1885-Cleveland.txt')+inaugural.words('1893-Cleveland.txt')+inaugural.words('1905-Roosevelt.txt')+inaugural.words('1933-Roosevelt.txt')+inaugural.words('1937-Roosevelt.txt')+inaugural.words('1941-Roosevelt.txt')+inaugural.words('1945-Roosevelt.txt')+inaugural.words('1913-Wilson.txt')+inaugural.words('1917-Wilson.txt')+inaugural.words('1949-Truman.txt')+inaugural.words('1961-Kennedy.txt')+inaugural.words('1965-Johnson.txt')+inaugural.words('1969-Nixon.txt')+inaugural.words('1973-Nixon.txt')+inaugural.words('1977-Carter.txt')+inaugural.words('1993-Clinton.txt')+inaugural.words('1997-Clinton.txt')+inaugural.words('2009-Obama.txt')+inaugural.words('2013-Obama.txt')+inaugural.words('2021-Biden.txt')

In [413]:
republican = inaugural.words('1841-Harrison.txt')+inaugural.words('1889-Harrison.txt')+inaugural.words('1861-Lincoln.txt')+inaugural.words('1865-Lincoln.txt')+inaugural.words('1869-Grant.txt')+inaugural.words('1873-Grant.txt')+inaugural.words('1877-Hayes.txt')+inaugural.words('1881-Garfield.txt')+inaugural.words('1897-McKinley.txt')+inaugural.words('1901-McKinley.txt')+inaugural.words('1905-Roosevelt.txt')+inaugural.words('1933-Roosevelt.txt')+inaugural.words('1937-Roosevelt.txt')+inaugural.words('1941-Roosevelt.txt')+inaugural.words('1945-Roosevelt.txt')+inaugural.words('1909-Taft.txt')+inaugural.words('1921-Harding.txt')+inaugural.words('1925-Coolidge.txt')+inaugural.words('1929-Hoover.txt')+inaugural.words('1953-Eisenhower.txt')+inaugural.words('1957-Eisenhower.txt')+inaugural.words('1969-Nixon.txt')+inaugural.words('1973-Nixon.txt')+inaugural.words('1981-Reagan.txt')+inaugural.words('1985-Reagan.txt')+inaugural.words('1989-Bush.txt')+inaugural.words('2001-Bush.txt')+inaugural.words('2005-Bush.txt')+inaugural.words('2017-Trump.txt') 

In [523]:
#remove stopwords 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
#remove punctuation
custom_stopwords = set((',',' ','.', ';', '?', '-', '!', '(', ')','--','"',"'", ':', '¡¦', '¡','', '9', '/', '11','ii', '400','1863','','people','shall','may','country','government','world', 'nation', 'must', 'great','upon','america','new','one','states','peace','every','us',',"'))

In [617]:
dem_filtered = [word.lower() for word in democrat if word.lower() not in stop_words] 

In [618]:
dem_filtered2 = [ word.lower() for word in dem_filtered if word.lower() not in custom_stopwords ] 

In [619]:
len(dem_filtered2)

21901

In [679]:
democrat1=[]
for i in range(43):
    democrat1.append([dem_filtered2[i*500:(i+1)*500],'dem'])

In [680]:
len(democrat1)

43

In [681]:
rep_filtered = [word.lower() for word in republican if word.lower() not in stop_words] 

In [682]:
rep_filtered2 = [ word.lower() for word in rep_filtered if word.lower() not in custom_stopwords ] 

In [683]:
len(rep_filtered2)

32344

In [684]:
republican1=[]
for i in range(64):
    republican1.append([rep_filtered2[i*500:(i+1)*500],'rep'])

In [685]:
len(republican1)

64

We now have a list of 24 Democrat and 35 Republican 1000-word segments.

In [686]:
fdist = nltk.FreqDist(dem_filtered2)
fdist.most_common(20) 

[('let', 109),
 ('time', 102),
 ('power', 90),
 ('nations', 78),
 ('life', 78),
 ('union', 74),
 ('american', 73),
 ('citizens', 72),
 ('would', 72),
 ('spirit', 72),
 ('today', 70),
 ('united', 69),
 ('men', 69),
 ('free', 68),
 ('fellow', 65),
 ('together', 65),
 ('know', 63),
 ('constitution', 62),
 ('work', 60),
 ('public', 59)]

In [687]:
fdist1 = nltk.FreqDist(rep_filtered2)
fdist1.most_common(20) 

[('freedom', 132),
 ('power', 129),
 ('citizens', 119),
 ('constitution', 116),
 ('time', 112),
 ('would', 111),
 ('law', 107),
 ('free', 106),
 ('make', 104),
 ('public', 103),
 ('american', 99),
 ('united', 96),
 ('congress', 93),
 ('made', 92),
 ('years', 92),
 ('nations', 92),
 ('war', 90),
 ('national', 88),
 ('men', 86),
 ('good', 85)]

In [688]:
all=dem_filtered2+rep_filtered2
all_words = nltk.FreqDist(w.lower() for w in all)
word_features = list(all_words)[:2000] 

wlist = []
for i in range(0, 2000, 200):
    df = pd.DataFrame(word_features[i:(i+200)])
    df.columns=['200 words']
    wlist.append(df)

#pd.concat(wlist, axis=1)

In [689]:
def document_features(document): 
    document_words = set(document) 
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [690]:
features = document_features(dem_filtered2)
list(features.items())[:20]

[('contains(fellow)', True),
 ('contains(citizens)', True),
 ('contains(american)', True),
 ('contains(expressed)', True),
 ('contains(unsolicited)', True),
 ('contains(suffrages)', True),
 ('contains(calls)', True),
 ('contains(pass)', True),
 ('contains(solemnities)', True),
 ('contains(preparatory)', True),
 ('contains(taking)', True),
 ('contains(duties)', True),
 ('contains(president)', True),
 ('contains(united)', True),
 ('contains(another)', True),
 ('contains(term)', True),
 ('contains(approbation)', True),
 ('contains(public)', True),
 ('contains(conduct)', True),
 ('contains(period)', True)]

In [691]:
documents=republican1+democrat1

In [704]:
random.shuffle(documents)
featuresets = [(document_features(d), c) for (d,c) in documents]
len(featuresets)

107

In [705]:
size = int(len(featuresets) * 0.2)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [706]:
len(test_set)

21

In [707]:
print(nltk.classify.accuracy(classifier, test_set))

0.7142857142857143


In [708]:
classifier.show_most_informative_features()

Most Informative Features
          contains(gave) = True              dem : rep    =      7.1 : 1.0
        contains(actual) = True              dem : rep    =      7.1 : 1.0
         contains(globe) = True              dem : rep    =      7.1 : 1.0
     contains(diversity) = True              dem : rep    =      5.4 : 1.0
  contains(achievements) = True              dem : rep    =      5.4 : 1.0
      contains(honestly) = True              dem : rep    =      5.4 : 1.0
          contains(calm) = True              dem : rep    =      4.6 : 1.0
       contains(hostile) = True              dem : rep    =      4.6 : 1.0
        contains(forget) = True              dem : rep    =      4.6 : 1.0
        contains(uphold) = True              dem : rep    =      4.6 : 1.0


In [673]:
type(test_set)

list

In [639]:
def show_most_informative_features_in_list(classifier, n=10):
    """
    Return a nested list of the "most informative" features 
    used by the classifier along with it's predominant labels
    """
    cpdist = classifier._feature_probdist       # probability distribution for feature values given labels
    feature_list = []
    for (fname, fval) in classifier.most_informative_features(n):
        def labelprob(l):
            return cpdist[l, fname].prob(fval)
        labels = sorted([l for l in classifier._labels if fval in cpdist[l, fname].samples()], 
                        key=labelprob)
        feature_list.append([fname, labels[-1]])
    return feature_list

In [573]:
show_most_informative_features_in_list(classifier,10)

[['contains(fruitful)', 'dem'],
 ['contains(compromise)', 'dem'],
 ['contains(appropriate)', 'dem'],
 ['contains(minds)', 'dem'],
 ['contains(concession)', 'dem'],
 ['contains(excited)', 'dem'],
 ['contains(opened)', 'dem'],
 ['contains(achievements)', 'dem'],
 ['contains(eyes)', 'dem'],
 ['contains(conferred)', 'dem']]